#### Setup

In [88]:
import sys
import numpy as np
import pandas as pd
import math
import random
from concorde.tsp import TSPSolver
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
print(sys.version)


3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 22:45:16) 
[Clang 9.0.1 ]


In [2]:
# GLOBAL VARIABLES
field_width = 100
field_height = 100
depot_x = 50
depot_y = 50

In [111]:
class Instance():
    
    def __init__(self, xlocs, ylocs, demands, solve_TSP=True):

        self.size = len(demands)-1
        self.demands = demands
        self.xlocs = xlocs
        self.ylocs = ylocs
        self.distances = self.calc_distance_matrix()
        self.optimal_routes = 'None'
        self.tour = 'None'
        if solve_TSP:
            self.tour = self.solve_TSP()
        
    def calc_distance_matrix(self):

        distances = np.zeros((self.size+1, self.size+1), dtype=float)
        for i in range(self.size+1):
            for j in range(self.size+1):
                new_dist = math.sqrt((self.xlocs[i]-self.xlocs[j])**2 + (self.ylocs[i]-self.ylocs[j])**2)
                distances[i,j] = new_dist
        return distances

    def get_lowerbound(self, capacity):
        return (2/capacity) * sum([self.demands[i]*self.distances[0,i]
                                        for i in range(len(self.demands))])
    
    def get_fleet_size(self, route_size):
        assert self.size % route_size == 0, "Number of customers must be evenly divisible by the route size."
        return int(self.size / route_size)
    
    def solve_TSP(self):
        solver = TSPSolver.from_data(self.xlocs, self.ylocs, 'EUC_2D')
        solution = solver.solve()
        
        return list(solution.tour)
    
    def save_optimal_routes(self, route_list):
        self.optimal_routes = route_list

In [13]:
def get_circular_cost(inst,segment):
    return sum([inst.distances[segment[i],segment[i+1]] for i in range(len(segment)-1)])

def get_radial_cost(inst,segment):
    """Assumes vehicle travels to/from the depot at segment endpoints."""
    return inst.distances[0,segment[0]] + inst.distances[0,segment[-1]]

def get_total_cost(inst,segment):
    return get_circular_cost(inst,segment)+get_radial_cost(inst,segment)

In [109]:
def solve_SDVRP(inst, capacity):
    
    def create_demand_points():
        return

    def create_data_model(inst, capacity):
        data = {}
        data['distance_matrix'] = inst.distances
        data['demands'] = inst.demands
        data['vehicle_capacities'] = [capacity]*inst.size
        data['num_vehicles'] = inst.size
        data['depot'] = 0
        return data

    def print_solution(data, manager, routing, solution):
        """Prints solution on console."""
        total_distance = 0
        total_load = 0
        for vehicle_id in range(data['num_vehicles']):
            index = routing.Start(vehicle_id)
            plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
            route_distance = 0
            route_load = 0
            while not routing.IsEnd(index):
                node_index = manager.IndexToNode(index)
                route_load += data['demands'][node_index]
                plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                route_distance += routing.GetArcCostForVehicle(
                    previous_index, index, vehicle_id)
            plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                     route_load)
            plan_output += 'Distance of the route: {}m\n'.format(route_distance)
            plan_output += 'Load of the route: {}\n'.format(route_load)
            print(plan_output)
            total_distance += route_distance
            total_load += route_load
        print('Total distance of all routes: {}m'.format(total_distance))
        print('Total load of all routes: {}'.format(total_load))

    def get_routes(solution, routing, manager):
        """Get vehicle routes from a solution and store them in an array."""
        # Get vehicle routes and store them in a two dimensional array whose
        # i,j entry is the jth location visited by vehicle i along its route.
        routes = []
        for route_nbr in range(routing.vehicles()):
            index = routing.Start(route_nbr)
            route = [manager.IndexToNode(index)]
            while not routing.IsEnd(index):
                index = solution.Value(routing.NextVar(index))
                route.append(manager.IndexToNode(index))
            routes.append(route)
        return routes

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    
    ###################
    ### RUN PROGRAM ###
    ###################
    
    # Zero cost if no demands
    if all(dem == 0 for dem in inst.demands):
        return (0, 0, 0)
    
    # Set up data model
    data = create_data_model(inst, capacity)
    
    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    
    # Create routing model
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add capacity constraint
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)
    all_routes = get_routes(solution, routing, manager)
    nonempty_routes = [route for route in all_routes if not all(i == 0 for i in route)]
    return nonempty_routes)
    #num_trips = sum(sum(d for d in r)>0 for r in routes)
    #return solution.ObjectiveValue(), end-start, num_trips
    """
    # Print solution on console.
    if solution:
        print('Solution found.')
        print('Objective: {}'.format(solution.ObjectiveValue()))
        #print_solution(data, manager, routing, solution)
        for i, route in enumerate(routes):
            print('Route', i, route)
    """

In [121]:
solve_SDVRP(inst,20)

[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 4, 7, 10, 5, 6, 0], [0, 2, 8, 3, 1, 9, 0]]

[[0, 4, 7, 10, 5, 6, 0], [0, 2, 8, 3, 1, 9, 0]]
336


In [119]:
def create_full_trips(inst, route_list, capacity):
    """Argument route_list is a list of lists of customer indices - e.g., [[1,2,3,4]] or [[1,2],[3,4]]"""
    
    assert type(route_list) == list, "route_list must be contained within square brackets as a list"
    
    segments = []
    for route in  route_list:
        i = 0
        new_seg = {}
        while i < len(route):
            cust = route[i]
            for d in range(1,inst.demands[cust]+1):
                if cust not in new_seg:
                    new_seg[cust] = 1
                else:
                    new_seg[cust]+=1
                if sum(new_seg.values()) == capacity:
                    seg_array = list(new_seg)
                    segments.append(seg_array)
                    new_seg = {}
            i+=1
        seg_array = list(new_seg)
        segments.append(seg_array)
    return segments

In [5]:
def get_dedicated_routes(inst, route_size):
    
    tour =  inst.tour[1:]
    routes = []
    for i in range(0,len(tour),route_size):
        new_route = tour[i:i+route_size]
        routes.append(new_route)
    return routes

In [117]:
N = 10
dmin = 0
dmax = 8
cust_x = field_width*np.random.random(N)
cust_y = field_height*np.random.random(N)
cust_dems = list(np.random.randint(dmin,dmax,N))
xlocs = list(np.append([depot_x], cust_x))
ylocs = list(np.append([depot_y], cust_y))
demands = list(np.append([0], cust_dems))
inst = Instance(xlocs, ylocs, demands)

In [85]:
capacity=20
route_size=5

In [118]:
print('Big tour:', inst.tour)

print('\n--- LOWERBOUND ---')
print('Lowerbound:', inst.get_lowerbound(capacity).round(1))

print('\n--- DEDICATED ---')
routes = get_dedicated_routes(inst, route_size)
trips = create_full_trips(inst,routes,capacity)
print('Routes:', *routes, sep="\n\t")
print('Trips:', *trips, sep="\n\t")
print('\nRadial cost:', sum([get_radial_cost(inst,seg) for seg in trips]).round(1))
print('Circular cost:', sum([get_circular_cost(inst,seg) for seg in trips]).round(1))
print('Total cost:', sum([get_total_cost(inst,seg) for seg in trips]).round(1))

print('\n-- Fully Flexible ---')
trips = create_full_trips(inst,[inst.tour],capacity)
print('Trips:', *trips, sep="\n\t")
print('\nRadial cost:', sum([get_radial_cost(inst,seg) for seg in trips]).round(1))
print('Circular cost:', sum([get_circular_cost(inst,seg) for seg in trips]).round(1))
print('Total cost:', sum([get_total_cost(inst,seg) for seg in trips]).round(1))

Big tour: [0, 9, 6, 1, 5, 10, 7, 3, 8, 2, 4]

--- LOWERBOUND ---
Lowerbound: 110.6

--- DEDICATED ---
Routes:
	[9, 6, 1, 5, 10]
	[7, 3, 8, 2, 4]
Trips:
	[9, 6, 1, 5, 10]
	[7, 3, 8, 2, 4]

Radial cost: 144.7
Circular cost: 207.0
Total cost: 351.7

-- Fully Flexible ---
Trips:
	[9, 6, 1, 5, 10, 7]
	[7, 3, 8, 2, 4]

Radial cost: 153.2
Circular cost: 230.9
Total cost: 384.1
